## 1 Dataframe  to predict/postprocess predictions only once

- do it for both verbs and nouns separately
with all possible options, merge the dataframes and then use it to make predictions

In [ ]:
from sesame.dataio import read_conll
from sesame.conll09 import CoNLL09Example, CoNLL09Element
from sesame.sentence import Sentence
import sys
import lexsub.conll_helper as conll_helper

from ordered_set import OrderedSet

data_dir = '../parser_workdir/data/open_sesame_v1_data/fn1.7'

exp = 'verbs'

dev_file = 'fn1.7.dev.syntaxnet.conll'
test_file = 'fn1.7.test.syntaxnet.conll'
train_file = 'fn1.7.fulltext.train.syntaxnet.conll'


input_file = f'{data_dir}/{exp}/{train_file}'

examples, __, __ = read_conll(input_file)

In [ ]:
from lexsub import augment_conll
from lexsub.augment_conll import PROC_FUNCS_OPTIONS, mask_sentences

parser = 'nltk' # for goldcluster
add_goldclusters = False
df1, config_dicts = mask_sentences(examples, 
                                   substitute_lu=True,
                                   substitute_role=True, role_tokens=[1], #role_postags='NN,NNS,NNP,NNPS',
                                   noun_max=1, ibn=True,
                                   parser=parser,
                                   add_goldclusters=add_goldclusters,
                                           verbose=False)

len(examples), len(config_dicts), len(df1)
# df1.head(10)

In [ ]:
df2, config_dicts2 = mask_sentences(examples, 
                                    substitute_lu=False,
                                    substitute_role=False, role_tokens=[1], #role_postags='NN,NNS,NNP,NNPS',
                                    noun_max=1, ibn=True,
                                    parser=parser,
                                    add_goldclusters=add_goldclusters,
                                           verbose=False)

len(examples), len(config_dicts2), len(df2)
# df2.head(10)

In [ ]:
df = df1.append(df2)
df['index'] = df['index'].apply(lambda x: x[0])
len(df)

### To generate predictions:

In [ ]:
dff = df.drop_duplicates(subset=['sentence', 'index']).reset_index(drop=True)
print(len(dff))

dff.to_pickle('../workdir/data/swv_fsp_T.pkl')


### To postprocess predictions:

In [ ]:
dff = df.drop_duplicates(subset=['sentence', 'index']).reset_index(drop=True)
df_pp = df.drop_duplicates(subset=['sentence', 'index', 'word_type']).reset_index(drop=True)
# for postprocessing you also need word_type
print(len(dff)), print(len(df_pp))
dff.head(5)

## 2. Read model predictions [only for post-processing once]

In [ ]:
import os
import pandas as pd
import pickle
from glob import glob
from pathlib import Path


results = '../workdir/results_parser/verb_preds_st'
df = pd.read_pickle('../workdir/data/swv_fsp_T.pkl').reset_index(drop=True)

# results = '../workdir/results_parser/noun_preds_st'
# df = pd.read_pickle('../workdir/data/swn_fsp_T.pkl').reset_index(drop=True)

# df['index'] = df['index'].apply(lambda x: x[0])
print(len(df))

subst_dir=results
model2res = {}
for p in Path(subst_dir).glob('*/results.csv'):
        print(p.parent.name)
        model2res[p.parent.name] = p
        
        

for p in Path(subst_dir).glob('*/predictions.pkl'):
#     print(p)
    print(p.parent.name)
    model2res[p.parent.name] = p


In [ ]:
models = [
# 'gie_sss_test_semiPURExlnet_embs', 
'gie_sss_test_semiPURExlnet_embs_swrhypers', 
# 'gie_sss_test_semiPURExlnet_embs_swnhypers', 
'gie_sss_test_semiPURExlnet_embs_swvhypers',
'blc-ntok1-nunits1-nomask-k200',
# 'bbc-ntok1-nunits1-nomask-k200',
# 'fasttext_cc_nolem',
# 'glove_840B_nolem',
# 'dt_wiki_lem',
# 'dt_59g_lem',
# 'melamud_baladd'
]

preds_dfs = {}

for k,v in model2res.items() :
    if not k in models: continue
    print(k)
    if str(v).endswith('.pkl'):
        df1 = df.copy()
        df1['pred_substitutes'] = pd.read_pickle(v)
        sdf = df1.copy()
    else:
        if not 'semiPURExlnet' in str(v): continue
        sdf = pd.read_csv(v, 
                  converters={'pred_substitutes':eval, 'context':eval})
    
        sdf['sentence'] = sdf.context.str.join('')
        sdf['st'] = sdf.apply(lambda r: len(''.join(r.context[:r.target_position])), axis=1)
        sdf['index'] = sdf.apply(lambda r: (r.st, r.st+len(r.context[r.target_position].strip())), axis=1)
        
    preds_dfs[k] = sdf

### 2.1 copy all predictions to one dataframe, if needs to post process, then use use dataframe from step#1

In [ ]:
df = df_pp.copy() 


In [ ]:
# Get for each input example its substitutes by merging dataframes on (index, sentence) key
def add_substs(idf, sdf):
    print(len(idf), len(sdf))
    mdf = idf.merge(sdf[['index','sentence','pred_substitutes']], how='left', on=['index','sentence'], 
                       validate='many_to_one')
    assert mdf.pred_substitutes.isnull().sum()==0 # check that all input examples received substitutes!
    return mdf


models = []
for k,v in preds_dfs.items():
    print(k)
    mdf = add_substs(df.copy(), v)
    df[k.replace('gie_sss_test_', '')] = mdf['pred_substitutes']
    models.append(k.replace('gie_sss_test_', ''))
#     break
# df['xlnet_embs_hypers'] = df.apply(lambda row: row['semiPURExlnet_embs_swnhypers'] if row['word_type']=='lu_n' else row['semiPURExlnet_embs_swrhypers'], axis=1)
df['xlnet_embs_hypers'] = df.apply(lambda row: row['semiPURExlnet_embs_swvhypers'] if row['word_type']=='lu_v' else row['semiPURExlnet_embs_swrhypers'], axis=1)
df = df.rename(columns={'blc-ntok1-nunits1-nomask-k200':'bert'})

# df['bert_best'] = df.apply(lambda row: row['blc-ntok1-nunits1-nomask-k200'] if row['word_type']=='lu_v' else row['bbc-ntok1-nunits1-mask-k200'], axis=1)
# df['bert_best'] = df.apply(lambda row: row['bbc-ntok1-nunits1-nomask-k200'] if row['word_type']=='lu_n' else row['bbc-ntok1-nunits1-mask-k200'], axis=1)

# df['dt'] = df.apply(lambda row: row['dt_59g_lem'] if row['word_type']=='lu' else row['dt_wiki_lem'], axis=1)
# df['embs'] = df.apply(lambda row: row['fasttext_cc_nolem'] if row['word_type']=='lu' else row['glove_840B_nolem'], axis=1)

models.append('xlnet_embs_hypers')
models.append('bert')
# models.append('dt')
# models.append('embs')



In [ ]:
# columns = ['sentence', 'masked_sent','index', 'seed_word', 'gold_cluster', 'word_type', 'postag', 'identifier', 'e_id', 'ex_id',
#           'bert', 'xlnet_embs_hypers', 'dt', 'embs','melamud_baladd']
columns = ['sentence', 'masked_sent','index', 'seed_word', 'gold_cluster', 'word_type', 'postag', 'identifier', 'e_id', 'ex_id',
          'bert', 'xlnet_embs_hypers']
df_post = df[columns].copy()
df_post

### 2.2 Post-process 


In [ ]:
models = [
 'bert',
 'xlnet_embs_hypers',
#  'dt',
#  'embs',
#  'melamud_baladd',
]


In [ ]:
PROC_FUNCS_OPTIONS={
'nolemma' : {
            'lu_v': 'clean_noisy,filter_vocab,match_inflection',
            'lu_n': 'clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns,match_inflection' ,
            'role' : 'clean_noisy',
            'noun':'clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns,match_inflection'
            },
    
'lemma' : {
            'lu_v': 'lemmatize,clean_noisy,filter_vocab',
            'lu_n': 'lemmatize,clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns',
            'role' : 'lemmatize,clean_noisy',
            'noun':'lemmatize,clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns'
            },

'nolemma_role_stopwords' : {
                    'lu_v': 'clean_noisy,filter_vocab,match_inflection',
                    'lu_n': 'clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns,match_inflection',
                    'role' : 'clean_noisy,remove_role_stopwords',
                    'noun':'clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns,match_inflection'
                    },
'lemma_role_stopwords' : {
                    'lu_v': 'lemmatize,clean_noisy, filter_vocab',
                    'lu_n': 'lemmatize,clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns',
                    'role' : 'lemmatize,clean_noisy,remove_role_stopwords',
                    'noun':'lemmatize,clean_noisy,remove_digits,remove_noun_stopwords,filter_nouns'
                    },
}

In [ ]:
temp_df = df_post.copy()    

In [ ]:
%load_ext autoreload
%autoreload 2
from lexsub.augment_conll import postprocess_predictions

for m in models:
    print(m)
    df_post[f'{m}_final_preds'] = postprocess_predictions(temp_df, df_post[f'{m}'].tolist(),
                                                    proc_funcs=PROC_FUNCS_OPTIONS['nolemma'],
                                                         verbose=True)
    
#     break


In [ ]:
df_post.to_pickle("../workdir/data/swv_nltk_nolemma_final_predictions.pkl")

In [ ]:
%load_ext autoreload
%autoreload 2
from lexsub.augment_conll import postprocess_predictions

for m in models:
    print(m)
    df_post[f'{m}_final_preds'] = postprocess_predictions(temp_df, df_post[f'{m}'].tolist(),
                                                    proc_funcs=PROC_FUNCS_OPTIONS['nolemma_role_stopwords'],
                                                         verbose=True)
    

In [ ]:
df_post.to_pickle("../workdir/data/swv_nltk_nolemma_role_stopwords_final_predictions.pkl")    

In [ ]:
df_post.loc[df_post['word_type']=='role']

## 3.1 Copy predictions to each expanded dataset folder

In [ ]:
models = [
 'bert',
 'xlnet_embs_hypers',
#  'dt',
#  'embs',
#  'melamud_baladd',
]


In [ ]:
import pandas as pd

data_prefix = 'swv'
parser = 'nltk'
# pipeline = 'nolemma'
pipeline = 'nolemma_role_stopwords'

predictions_file = f'{data_prefix}_{parser}_{pipeline}_final_predictions.pkl'

df_post = pd.read_pickle(f'../workdir/data/{predictions_file}')

src_df = df_post.copy()
# just need predictions, regardless of word_type
src_df = src_df.drop_duplicates(subset=['sentence', 'index']).reset_index(drop=True)

src_df.head(5)

In [ ]:
import os
import pickle
import pandas as pd
from glob import glob
# Get for each input example its substitutes by merging dataframes on (index, sentence) key
def add_substs(idf, sdf, preds_column):
#     print(len(idf)), print(len(sdf))
    mdf = idf.merge(sdf, how='left', on=['index','sentence'], 
                       validate='many_to_one')
    assert mdf[preds_column].isnull().sum()==0 # check that all input examples received substitutes!
    return mdf

# exps_dir = 'expanded_nPc_verbs_randAllExps'
exps_dir = 'expanded_nExPerSent_verbs_randAllExps'
# exps_dir = 'expanded_nPc_nouns_randAllExps'
# exps_dir = 'expanded_nExPerSent_nouns_randAllExps'
exps = '*'
for f in glob(f'../parser_workdir/data/open_sesame_v1_data/fn1.7/{exps_dir}/{exps}/data.pkl'):
    if not 'roles' in f: continue
    print(f)
    path = f.replace('/data.pkl', '')
    idf= pd.read_pickle(f)
    idf['index']=idf['index'].str[0]  # need hashable tuple instead of non-hashable list
    for m in models:
        print(m)
        preds_column = m
        sdf = src_df[['index','sentence', preds_column]].copy()
        mdf = add_substs(idf.copy(), sdf, preds_column)
        predictions = mdf[preds_column].tolist() 
#         break
#     break     

        dir_path = f'{path}/{m}'
        if not os.path.exists(dir_path):
            os.mkdir(dir_path) 
        
        with open(f"{dir_path}/predictions.pkl", 'wb') as fp:
            pickle.dump(predictions, fp)
            
            
#     break # remote this break to load all input dataframes

## 3.2 Copy final_predictions to each expanded dataset folder

In [ ]:
import pandas as pd

data_prefix = 'swv'
parser = 'nltk'
# pipeline = 'nolemma'
pipeline = 'nolemma_role_stopwords'

predictions_file = f'{data_prefix}_{parser}_{pipeline}_final_predictions.pkl'
print(predictions_file)
df_post = pd.read_pickle(f'../workdir/data/{predictions_file}')

src_df = df_post.copy()
column_postfix = "final_preds"
src_df.head(5)

In [ ]:
# Get for each input example its substitutes by merging dataframes on (index, sentence) key
def add_substs(idf, sdf, preds_column):
    mdf = idf.merge(sdf, how='left', on=['word_type', 'index','sentence'], 
                       validate='many_to_one')
    assert mdf[preds_column].isnull().sum()==0 # check that all input examples received substitutes!
    return mdf


# exps_dir = 'expanded_nPc_verbs_randAllExps'
exps_dir = 'expanded_nExPerSent_verbs_randAllExps'
# exps_dir = 'expanded_nPc_nouns_randAllExps'
# exps_dir = 'expanded_nExPerSent_nouns_randAllExps'
exps = '*'

for f in glob(f'../parser_workdir/data/open_sesame_v1_data/fn1.7/{exps_dir}/{exps}/data.pkl'):
#     if not 'roles' in f: continue
    print(f.replace('../parser_workdir/data/open_sesame_v1_data/fn1.7/', ''))
    path = f.replace('/data.pkl', '')
    idf=pd.read_pickle(f)
    idf['index']=idf['index'].str[0]  # need hashable tuple instead of non-hashable list
    for m in models:
        print(m)
        preds_column = f'{m}_{column_postfix}'
        sdf = src_df[['word_type', 'index','sentence', preds_column]].copy()
        mdf = add_substs(idf.copy(), sdf, preds_column)
        predictions = mdf[preds_column].tolist() 
        #break    
        dir_path = f'{path}/{m}'
        if not os.path.exists(dir_path):
            os.mkdir(dir_path) 
        
        with open(f"{dir_path}/{parser}_{pipeline}_final_predictions.pkl", 'wb') as fp:
            pickle.dump(predictions, fp)
